#### Learning quantization

Quatization is the process of converting large set of values into smaller one

We would be quantizing hugging face models so that we can run them on low memory


Install below libraries

In [2]:
!pip install transformers
!pip install quanto
!pip install torch

  Using cached ninja-1.11.1.1-py2.py3-none-manylinux1_x86_64.manylinux_2_5_x86_64.whl.metadata (5.3 kB)
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-no

EleutherAI is a non profit research lab, focused on interpretability, alignment, and ethics of AI. We would be importing this model from huggingface.
What is hugging face?

In [3]:
from transformers import AutoModelForCausalLM
model_name = "EleutherAI/pythia-410m"

/home/aaekay/miniconda3/envs/test2/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


What is AutoModelForCausaLM? <br>
HF provied a class where it automatically infers from the model name that what kind of config it would need

In [5]:
model = AutoModelForCausalLM.from_pretrained(model_name, low_cpu_mem_usage = False) # to use low cpu mem usage we require accelerate,
# we are keeping it false since we dont have ram limitations for now

autotokenizer class would help in loading the tokenizer

In [6]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [7]:
text = "Hello my name is bond, James "
inputs = tokenizer(text, return_tensors="pt")
outputs = model.generate(**inputs, max_new_tokens=10)
print(outputs)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


tensor([[12092,   619,  1416,   310,  5533,    13,  5490,   209,   187,    42,
          1353,   247,   747, 17782,   281,   253,  2165,    15]])
Hello my name is bond, James 
I'm a newbie to the game.


Let's estimate the size of the model \
it is 410 Million Parameter with fp32, it would be 4 bytes for each resulting in 1.6 gb of model. \
The below function will help us to get the size

In [9]:
import torch
def named_module_tensors(module, recurse=False):
    for named_parameter in module.named_parameters(recurse=recurse):
      name, val = named_parameter
      flag = True
      if hasattr(val,"_data") or hasattr(val,"_scale"):
        if hasattr(val,"_data"):
          yield name + "._data", val._data
        if hasattr(val,"_scale"):
          yield name + "._scale", val._scale
      else:
        yield named_parameter

    for named_buffer in module.named_buffers(recurse=recurse):
      yield named_buffer

def dtype_byte_size(dtype):
    """
    Returns the size (in bytes) occupied by one parameter of type `dtype`.
    """
    import re
    if dtype == torch.bool:
        return 1 / 8
    bit_search = re.search(r"[^\d](\d+)$", str(dtype))
    if bit_search is None:
        raise ValueError(f"`dtype` is not a valid dtype: {dtype}.")
    bit_size = int(bit_search.groups()[0])
    return bit_size // 8
def compute_module_sizes(model):
    """
    Compute the size of each submodule of a given model.
    """
    from collections import defaultdict
    module_sizes = defaultdict(int)
    for name, tensor in named_module_tensors(model, recurse=True):
      size = tensor.numel() * dtype_byte_size(tensor.dtype)
      name_parts = name.split(".")
      for idx in range(len(name_parts) + 1):
        module_sizes[".".join(name_parts[:idx])] += size

    return module_sizes

In [14]:
model_size = compute_module_sizes(model)
print(model_size[""]*1e-9, "GB")

1.6402112960000002 GB


In [19]:
from quanto import freeze, quantize
# quantize(model, weights=torch.int8, activations=None) # for older qunato it would work
import quanto
print(quanto.__version__)
# version 0.2 uses quanto.qint8
quantize(model, weights=quanto.qint8, activations=None)
# call freeze it, the float weights would be replace by linear weights
freeze(model)


0.2.0


In [20]:
model_size = compute_module_sizes(model)
print(model_size[""]*1e-9, "GB")

0.580794472 GB


Let's check the output of the model


In [21]:
outputs = model.generate(**inputs, max_new_tokens=10)
print(outputs)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variabl

tensor([[12092,   619,  1416,   310,  5533,    13,  5490,   209,   187,   395,
           309,   717,   247,   747, 17782,   275,   253,  2165]])
Hello my name is bond, James 
and I am a newbie in the game
